# AILib Tutorial 5: Session Management

Sessions in AILib help you maintain conversation state across multiple interactions. In this tutorial, you'll learn:

- Creating and managing sessions
- Storing conversation history
- Managing session metadata
- Session persistence
- Advanced session patterns
- Best practices for stateful conversations

## Setup

Let's import what we need:

In [ ]:
from ailib import OpenAIClient, Session
from ailib.prompts import PromptTemplate, PromptBuilder
from dotenv import load_dotenv
import json
from datetime import datetime

# Load environment variables
load_dotenv()

# Create a client
client = OpenAIClient()
print("Ready to manage sessions!")

## Basic Session Usage

Sessions store conversation history and metadata:

In [ ]:
# Create a new session
session = Session()

# Add messages to the session
session.add_message("system", "You are a helpful assistant.")
session.add_message("user", "What's the weather like?")
session.add_message("assistant", "I don't have access to real-time weather data.")

# Get all messages
messages = session.get_messages()
print("Session messages:")
for msg in messages:
    print(f"- {msg['role']}: {msg['content'][:50]}...")

# Get session info
print(f"\nSession ID: {session.session_id}")
print(f"Message count: {len(session)}")

## Using Sessions with LLM Clients

Sessions integrate seamlessly with LLM clients:

In [ ]:
# Create a session for a coding assistant
coding_session = Session()
coding_session.add_message("system", "You are a Python coding assistant.")

# Function to chat with session
def chat_with_session(session, user_input, client):
    """Add user input to session and get response."""
    # Add user message
    session.add_message("user", user_input)
    
    # Get response from LLM
    messages = session.get_messages()
    response = client.chat(messages)
    
    # Add assistant response to session
    session.add_message("assistant", response)
    
    return response

# Have a conversation
response1 = chat_with_session(
    coding_session, 
    "How do I read a CSV file in Python?",
    client
)
print("Response 1:")
print(response1)

print("\n" + "="*50 + "\n")

# Continue the conversation
response2 = chat_with_session(
    coding_session,
    "How do I filter rows where the 'age' column is greater than 30?",
    client
)
print("Response 2 (with context):")
print(response2)

## Session Metadata

Store additional information with your sessions:

In [ ]:
# Create a session with metadata
customer_session = Session()

# Set metadata
customer_session.set_metadata("user_id", "12345")
customer_session.set_metadata("user_name", "John Doe")
customer_session.set_metadata("account_type", "premium")
customer_session.set_metadata("session_purpose", "technical_support")
customer_session.set_metadata("created_at", datetime.now().isoformat())

# Add system message with context
system_prompt = f"""You are a technical support agent.
Customer: {customer_session.get_metadata('user_name')}
Account Type: {customer_session.get_metadata('account_type')}
Be helpful and professional."""

customer_session.add_message("system", system_prompt)

# Retrieve metadata
print("Session Metadata:")
all_metadata = customer_session.get_all_metadata()
for key, value in all_metadata.items():
    print(f"  {key}: {value}")

# Use metadata in responses
response = chat_with_session(
    customer_session,
    "I'm having trouble with my API key",
    client
)
print(f"\nSupport Response for {customer_session.get_metadata('user_name')}:")
print(response)

## Session Persistence

Save and load sessions for long-term storage:

In [ ]:
# Create a session with some history
persistent_session = Session()
persistent_session.add_message("system", "You are a writing assistant.")
persistent_session.add_message("user", "Help me write a blog post about AI")
persistent_session.add_message("assistant", "I'd be happy to help you write a blog post about AI...")
persistent_session.set_metadata("topic", "AI blog post")
persistent_session.set_metadata("word_count_target", 1000)

# Save session to file
session_data = persistent_session.to_dict()
with open("session_backup.json", "w") as f:
    json.dump(session_data, f, indent=2)

print("Session saved!")
print(f"Session ID: {persistent_session.session_id}")
print(f"Messages: {len(persistent_session)}")

# Load session from file
with open("session_backup.json", "r") as f:
    loaded_data = json.load(f)

loaded_session = Session.from_dict(loaded_data)

print("\nSession loaded!")
print(f"Loaded Session ID: {loaded_session.session_id}")
print(f"Loaded Messages: {len(loaded_session)}")
print(f"Loaded Metadata: {loaded_session.get_metadata('topic')}")

# Continue the conversation
response = chat_with_session(
    loaded_session,
    "What should the introduction focus on?",
    client
)
print(f"\nContinued conversation:")
print(response[:200] + "...")

## Managing Long Conversations

Handle token limits and conversation length:

In [ ]:
class ManagedSession:
    """Session with automatic management features."""
    
    def __init__(self, max_messages=20, system_prompt=""):
        self.session = Session()
        self.max_messages = max_messages
        if system_prompt:
            self.session.add_message("system", system_prompt)
        self.system_prompt = system_prompt
    
    def add_exchange(self, user_msg, assistant_msg):
        """Add a user-assistant exchange."""
        self.session.add_message("user", user_msg)
        self.session.add_message("assistant", assistant_msg)
        
        # Check if we need to truncate
        if len(self.session) > self.max_messages:
            self._truncate_history()
    
    def _truncate_history(self):
        """Keep only recent messages plus system prompt."""
        messages = self.session.get_messages()
        
        # Keep system message if exists
        new_messages = []
        if messages[0]['role'] == 'system':
            new_messages.append(messages[0])
        
        # Keep last N messages
        keep_count = self.max_messages - len(new_messages)
        new_messages.extend(messages[-keep_count:])
        
        # Create new session with truncated history
        self.session = Session()
        for msg in new_messages:
            self.session.add_message(msg['role'], msg['content'])
        
        # Add truncation notice
        self.session.set_metadata("truncated", True)
        self.session.set_metadata("truncated_at", datetime.now().isoformat())
    
    def get_messages(self):
        """Get current messages."""
        return self.session.get_messages()
    
    def get_summary(self):
        """Get session summary."""
        return {
            "message_count": len(self.session),
            "truncated": self.session.get_metadata("truncated", False),
            "session_id": self.session.session_id
        }

# Use managed session
managed = ManagedSession(
    max_messages=6,
    system_prompt="You are a helpful assistant. Be concise."
)

# Simulate a long conversation
exchanges = [
    ("What is Python?", "Python is a high-level programming language."),
    ("What can I build with it?", "Web apps, data analysis tools, AI models, and more."),
    ("How do I start learning?", "Start with Python basics: variables, functions, and control flow."),
    ("What IDE should I use?", "VS Code, PyCharm, or Jupyter notebooks are popular choices."),
    ("What about packages?", "Use pip to install packages from PyPI."),
]

for user_msg, assistant_msg in exchanges:
    managed.add_exchange(user_msg, assistant_msg)
    print(f"After exchange {len(managed.session)//2}: {managed.get_summary()}")

print("\nFinal messages in session:")
for msg in managed.get_messages():
    print(f"- {msg['role']}: {msg['content'][:40]}...")

## Session Analytics

Track and analyze session metrics:

In [ ]:
class AnalyticsSession(Session):
    """Session with built-in analytics."""
    
    def __init__(self):
        super().__init__()
        self.start_time = datetime.now()
        self.set_metadata("start_time", self.start_time.isoformat())
    
    def add_message(self, role, content):
        """Add message with timestamp."""
        super().add_message(role, content)
        
        # Track message metrics
        timestamp = datetime.now()
        message_count = len(self.messages)
        
        # Store analytics
        self.set_metadata(f"msg_{message_count}_timestamp", timestamp.isoformat())
        self.set_metadata(f"msg_{message_count}_length", len(content))
    
    def get_analytics(self):
        """Get session analytics."""
        current_time = datetime.now()
        duration = (current_time - self.start_time).total_seconds()
        
        # Calculate metrics
        user_messages = [m for m in self.messages if m['role'] == 'user']
        assistant_messages = [m for m in self.messages if m['role'] == 'assistant']
        
        total_user_chars = sum(len(m['content']) for m in user_messages)
        total_assistant_chars = sum(len(m['content']) for m in assistant_messages)
        
        return {
            "session_id": self.session_id,
            "duration_seconds": duration,
            "total_messages": len(self.messages),
            "user_messages": len(user_messages),
            "assistant_messages": len(assistant_messages),
            "avg_user_msg_length": total_user_chars / max(len(user_messages), 1),
            "avg_assistant_msg_length": total_assistant_chars / max(len(assistant_messages), 1),
            "messages_per_minute": len(self.messages) / max(duration / 60, 1)
        }

# Use analytics session
analytics_session = AnalyticsSession()
analytics_session.add_message("system", "You are a helpful assistant.")

# Simulate conversation
conversation = [
    ("What is machine learning?", "Machine learning is a subset of AI..."),
    ("Give me an example", "Image recognition is a common example..."),
    ("How does it work?", "It works by training models on data...")
]

for user_msg, assistant_msg in conversation:
    analytics_session.add_message("user", user_msg)
    analytics_session.add_message("assistant", assistant_msg)

# Get analytics
analytics = analytics_session.get_analytics()
print("Session Analytics:")
for key, value in analytics.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.2f}")
    else:
        print(f"  {key}: {value}")

## Multi-User Sessions

Manage sessions for multiple users:

In [ ]:
class SessionManager:
    """Manages multiple user sessions."""
    
    def __init__(self):
        self.sessions = {}
        self.session_metadata = {}
    
    def create_session(self, user_id, **metadata):
        """Create a new session for a user."""
        session = Session()
        
        # Set user metadata
        session.set_metadata("user_id", user_id)
        for key, value in metadata.items():
            session.set_metadata(key, value)
        
        # Store session
        self.sessions[user_id] = session
        self.session_metadata[user_id] = {
            "created_at": datetime.now(),
            "last_accessed": datetime.now()
        }
        
        return session
    
    def get_session(self, user_id):
        """Get or create session for user."""
        if user_id not in self.sessions:
            return self.create_session(user_id)
        
        # Update last accessed
        self.session_metadata[user_id]["last_accessed"] = datetime.now()
        return self.sessions[user_id]
    
    def end_session(self, user_id):
        """End a user's session."""
        if user_id in self.sessions:
            session = self.sessions[user_id]
            # Could save to database here
            del self.sessions[user_id]
            del self.session_metadata[user_id]
            return True
        return False
    
    def get_active_sessions(self):
        """Get information about active sessions."""
        return [
            {
                "user_id": user_id,
                "message_count": len(session),
                "created_at": self.session_metadata[user_id]["created_at"],
                "last_accessed": self.session_metadata[user_id]["last_accessed"]
            }
            for user_id, session in self.sessions.items()
        ]

# Use session manager
manager = SessionManager()

# Create sessions for different users
users = [
    ("user_123", {"name": "Alice", "role": "developer"}),
    ("user_456", {"name": "Bob", "role": "designer"}),
    ("user_789", {"name": "Charlie", "role": "manager"})
]

for user_id, metadata in users:
    session = manager.create_session(user_id, **metadata)
    session.add_message("system", f"You are assisting {metadata['name']}, a {metadata['role']}.")
    session.add_message("user", "Hello!")
    session.add_message("assistant", f"Hello {metadata['name']}! How can I help you today?")

# Show active sessions
print("Active Sessions:")
for session_info in manager.get_active_sessions():
    print(f"  User: {session_info['user_id']}")
    print(f"    Messages: {session_info['message_count']}")
    print(f"    Created: {session_info['created_at'].strftime('%H:%M:%S')}")
    print()

## Session Templates

Create reusable session templates:

In [ ]:
class SessionTemplate:
    """Template for creating pre-configured sessions."""
    
    @staticmethod
    def coding_assistant(language="Python"):
        """Create a coding assistant session."""
        session = Session()
        session.add_message(
            "system",
            f"You are an expert {language} developer. Provide clear, "
            f"well-commented code examples and explain concepts thoroughly."
        )
        session.set_metadata("template", "coding_assistant")
        session.set_metadata("language", language)
        return session
    
    @staticmethod
    def customer_support(company_name, product):
        """Create a customer support session."""
        session = Session()
        session.add_message(
            "system",
            f"You are a helpful customer support agent for {company_name}. "
            f"You're assisting with {product}. Be professional, empathetic, "
            f"and provide clear solutions."
        )
        session.set_metadata("template", "customer_support")
        session.set_metadata("company", company_name)
        session.set_metadata("product", product)
        return session
    
    @staticmethod
    def creative_writing(genre, style):
        """Create a creative writing session."""
        session = Session()
        session.add_message(
            "system",
            f"You are a creative writing assistant specializing in {genre}. "
            f"Write in a {style} style. Help with plot development, "
            f"character creation, and prose improvement."
        )
        session.set_metadata("template", "creative_writing")
        session.set_metadata("genre", genre)
        session.set_metadata("style", style)
        return session

# Create sessions from templates
coding_session = SessionTemplate.coding_assistant("JavaScript")
support_session = SessionTemplate.customer_support("TechCorp", "CloudAPI")
writing_session = SessionTemplate.creative_writing("sci-fi", "descriptive")

# Use a template session
response = chat_with_session(
    coding_session,
    "How do I handle async/await errors?",
    client
)
print("Coding Assistant Response:")
print(response[:300] + "...")

print("\n" + "="*50 + "\n")

# Check session metadata
print("Session Metadata:")
for key, value in coding_session.get_all_metadata().items():
    print(f"  {key}: {value}")

## Real-World Example: Chatbot with Memory

Build a chatbot that remembers user preferences:

In [ ]:
class MemoryBot:
    """Chatbot with persistent memory across sessions."""
    
    def __init__(self, user_id, client):
        self.user_id = user_id
        self.client = client
        self.session = Session()
        self.memory = self._load_memory()
        self._initialize_session()
    
    def _load_memory(self):
        """Load user memory from storage."""
        # In production, load from database
        return {
            "preferences": {},
            "facts": [],
            "topics_discussed": []
        }
    
    def _save_memory(self):
        """Save user memory to storage."""
        # In production, save to database
        pass
    
    def _initialize_session(self):
        """Initialize session with memory context."""
        memory_context = self._format_memory()
        
        system_prompt = f"""You are a helpful assistant with memory.
        
User Memory:
{memory_context}

Remember new information shared by the user and reference past conversations naturally."""
        
        self.session.add_message("system", system_prompt)
    
    def _format_memory(self):
        """Format memory for context."""
        if not any(self.memory.values()):
            return "No previous interactions."
        
        lines = []
        if self.memory["preferences"]:
            lines.append("Preferences:")
            for key, value in self.memory["preferences"].items():
                lines.append(f"  - {key}: {value}")
        
        if self.memory["facts"]:
            lines.append("\nKnown facts:")
            for fact in self.memory["facts"][-5:]:  # Last 5 facts
                lines.append(f"  - {fact}")
        
        return "\n".join(lines)
    
    def chat(self, message):
        """Chat with the bot."""
        # Add user message
        self.session.add_message("user", message)
        
        # Get response
        messages = self.session.get_messages()
        response = self.client.chat(messages)
        
        # Add assistant response
        self.session.add_message("assistant", response)
        
        # Extract and store memory
        self._extract_memory(message, response)
        
        return response
    
    def _extract_memory(self, user_msg, bot_response):
        """Extract information to remember."""
        # Simple extraction - in production, use NLP
        user_lower = user_msg.lower()
        
        # Extract preferences
        if "i like" in user_lower or "i prefer" in user_lower:
            # Simple extraction
            if "python" in user_lower:
                self.memory["preferences"]["programming_language"] = "Python"
            if "coffee" in user_lower:
                self.memory["preferences"]["beverage"] = "coffee"
        
        # Extract facts
        if "my name is" in user_lower:
            # Extract name (simplified)
            parts = user_msg.split("my name is")
            if len(parts) > 1:
                name = parts[1].strip().split()[0]
                self.memory["facts"].append(f"User's name is {name}")
        
        # Track topics
        if "python" in user_lower:
            self.memory["topics_discussed"].append("Python programming")
        
        self._save_memory()

# Create and use memory bot
bot = MemoryBot("user_123", client)

# First conversation
print("Conversation 1:")
response1 = bot.chat("Hi! My name is Alex and I like Python programming.")
print(f"Bot: {response1}")

print("\n" + "="*50 + "\n")

# Bot should remember
print("Conversation 2 (bot should remember):")
response2 = bot.chat("What's my favorite programming language?")
print(f"Bot: {response2}")

# Check memory
print("\nBot's Memory:")
print(json.dumps(bot.memory, indent=2))

## Best Practices

Tips for effective session management:

In [ ]:
# 1. Always set appropriate metadata
def create_documented_session(purpose, user_info):
    session = Session()
    session.set_metadata("purpose", purpose)
    session.set_metadata("created_at", datetime.now().isoformat())
    session.set_metadata("user_info", user_info)
    session.set_metadata("version", "1.0")
    return session

# 2. Implement session cleanup
class AutoCleanupSession(Session):
    def __init__(self, ttl_minutes=60):
        super().__init__()
        self.ttl_minutes = ttl_minutes
        self.created_at = datetime.now()
    
    def is_expired(self):
        age = (datetime.now() - self.created_at).total_seconds() / 60
        return age > self.ttl_minutes

# 3. Handle sensitive data carefully
def sanitize_session(session):
    """Remove sensitive data before storage."""
    sensitive_patterns = ["password", "api_key", "secret"]
    
    messages = session.get_messages()
    for msg in messages:
        for pattern in sensitive_patterns:
            if pattern in msg['content'].lower():
                msg['content'] = "[REDACTED]"
    
    return session

# 4. Implement session versioning
class VersionedSession(Session):
    def __init__(self):
        super().__init__()
        self.version_history = []
    
    def checkpoint(self, description=""):
        """Create a version checkpoint."""
        checkpoint = {
            "timestamp": datetime.now().isoformat(),
            "description": description,
            "messages": self.messages.copy(),
            "metadata": self.metadata.copy()
        }
        self.version_history.append(checkpoint)
    
    def restore(self, version_index):
        """Restore to a previous version."""
        if 0 <= version_index < len(self.version_history):
            checkpoint = self.version_history[version_index]
            self.messages = checkpoint["messages"].copy()
            self.metadata = checkpoint["metadata"].copy()

print("Best practices examples created!")

## Summary

In this tutorial, you learned:

- ✅ How to create and manage sessions
- ✅ How to store and retrieve conversation history
- ✅ How to use session metadata effectively
- ✅ How to persist sessions for long-term storage
- ✅ How to handle long conversations and token limits
- ✅ How to build multi-user session systems
- ✅ How to create reusable session templates
- ✅ Best practices for session management

Sessions are essential for:
- Building stateful AI applications
- Maintaining conversation context
- Tracking user interactions
- Creating personalized experiences

## Next Steps

Continue your journey with:

- **Tutorial 6: Chains** - Connect multiple operations sequentially
- **Tutorial 7: Tools and Decorators** - Extend agent capabilities
- **Tutorial 8: Agents** - Build autonomous AI agents

Happy session management! 💬